In [2]:

from utils.prompt import template
from utils.utils import process_pdfs_from_dataframe, process_pdfs_from_dataframe_faiss
import os 
import string
import pandas as pd 
from tqdm.autonotebook import tqdm

from utils.RagPipeline import Ragpipeline

from utils.utils import normalize_path
from utils.cal import calculate_average_f1_score

## Dense DB (Vector DB) 및 Sparse DB

test.csv에서 질문들에 매칭되는 소스(pdf) 정보가 주어진다. 

각 질문에 따른 소스에 접근을 하고 리트리버를 사용해도 되기 때문에 

각 질문에 따른 소스별 DB를 만들어주자

In [3]:
df = pd.read_csv('../open/train.csv')
base_directory = '../open' # Your Base Directory

df.head()

,SAMPLE_ID,Source,Source_path,Question,Answer
0,TRAIN_000,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?,"2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,TRAIN_001,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,TRAIN_002,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,기금이 예산과 다른 점은?,"기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,TRAIN_003,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"일반회계, 특별회계, 기금 간의 차이점은 무엇인가요?","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,TRAIN_004,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요?","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."


In [4]:
source_list = list(df['Source'].unique())

# 알파벳 리스트
alphabet = list(string.ascii_uppercase)

In [5]:
# 딕셔너리로 파일명과 알파벳 매핑
file_mapping = {alphabet[i]: source_list[i] for i in range(len(source_list))}

reversed_file_mapping = {value: key for key, value in file_mapping.items()}

file_mapping

{'A': '1-1 2024 주요 재정통계 1권',
 'B': '2024 나라살림 예산개요',
 'C': '재정통계해설',
 'D': '국토교통부_전세임대(융자)',
 'E': '고용노동부_청년일자리창출지원',
 'F': '고용노동부_내일배움카드(일반)',
 'G': '보건복지부_노인일자리 및 사회활동지원',
 'H': '중소벤처기업부_창업사업화지원',
 'I': '보건복지부_생계급여',
 'J': '국토교통부_소규모주택정비사업',
 'K': '국토교통부_민간임대(융자)',
 'L': '고용노동부_조기재취업수당',
 'M': '2024년도 성과계획서(총괄편)',
 'N': '「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》',
 'O': '「FIS 이슈 & 포커스」 22-3호 《재정융자사업》',
 'P': '월간 나라재정 2023년 12월호'}

In [5]:
# # PDF별 FAISS db 생성 
# from config import config
# process_pdfs_from_dataframe_faiss(config, df, base_directory, reversed_file_mapping, save_path='./train_data')

가지고 있는 pdf파일들: ['./train_source/1-1 2024 주요 재정통계 1권.pdf'
 './train_source/2024 나라살림 예산개요.pdf' './train_source/재정통계해설.pdf'
 './train_source/국토교통부_전세임대(융자).pdf' './train_source/고용노동부_청년일자리창출지원.pdf'
 './train_source/고용노동부_내일배움카드(일반).pdf'
 './train_source/보건복지부_노인일자리 및 사회활동지원.pdf'
 './train_source/중소벤처기업부_창업사업화지원.pdf' './train_source/보건복지부_생계급여.pdf'
 './train_source/국토교통부_소규모주택정비사업.pdf' './train_source/국토교통부_민간임대(융자).pdf'
 './train_source/고용노동부_조기재취업수당.pdf' './train_source/2024년도 성과계획서(총괄편).pdf'
 './train_source/「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》.pdf'
 './train_source/「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf'
 './train_source/월간 나라재정 2023년 12월호.pdf']


Processing PDFs:   0%|          | 0/16 [00:00<?, ?it/s]

Processing 1-1 2024 주요 재정통계 1권...


c:\Users\kim_h\anaconda3\envs\final\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Processing PDFs:   6%|▋         | 1/16 [00:23<05:53, 23.60s/it]

Processing 2024 나라살림 예산개요...


...

Processing PDFs:  88%|████████▊ | 14/16 [03:07<00:31, 15.55s/it]

Processing 「FIS 이슈 & 포커스」 22-3호 《재정융자사업》...


Processing PDFs:  94%|█████████▍| 15/16 [03:12<00:12, 12.59s/it]

Processing 월간 나라재정 2023년 12월호...


Processing PDFs: 100%|██████████| 16/16 [03:30<00:00, 13.16s/it]


In [6]:
source = source_list[0]

target_df = df[df['Source'] == source][:]

In [7]:
source

'1-1 2024 주요 재정통계 1권'

In [8]:
len(target_df)

50

In [10]:
from config import config

In [11]:
pipeline = Ragpipeline(f'train_data/{reversed_file_mapping[source]}', config)

# 결과를 저장할 리스트 초기화
results = []

for _, row in tqdm(target_df.iterrows(), total=len(target_df), desc=f"Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_path(row['Source'])
    
    # 질문 
    question = row['Question']
    print(f"Question: {question}")
    
    # 대답
    full_response = pipeline.answer_generation(question)
    first_line_response = full_response.split("\n")[0]
    print(f"Answer: {first_line_response}")
    
    # 정답 
    print(f"GT: {row['Answer']}\n")
    
    # 결과 저장
    results.append({
        "Answer": first_line_response,
        "GT": row['Answer']
    })
    
# results 리스트를 DataFrame으로 변환
df_results = pd.DataFrame(results)
pred = df_results['Answer']
gt = df_results['GT']

f1_score = calculate_average_f1_score(gt, pred)
print(f1_score)

# 데이터프레임 출력
df_results.head()

c:\Users\kim_h\anaconda3\envs\final\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Answering Questions:   0%|          | 0/50 [00:00<?, ?it/s]

Question: 2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?


Answering Questions:   2%|▏         | 1/50 [00:30<25:16, 30.94s/it]

Answer: 2024년 기준으로 중앙정부 재정체계는 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.
GT: 2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.

Question: 2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?


Answering Questions:   4%|▍         | 2/50 [00:44<16:41, 20.86s/it]

...

Answering Questions:  98%|█████████▊| 49/50 [13:45<00:14, 14.77s/it]

Answer: 총사업비 관리대상사업의 규모는 2023년 286.2조원이며, 전년 대비 7.3조원 증가했습니다.
GT: 총사업비 관리대상사업의 규모는 305.0조원이며, 전년 대비 30.7조원 증가했습니다.

Question: 총사업비 관리제도는 어떤 제도이며, 어떤 종류의 사업을 관리하는데 사용되나요?


Answering Questions: 100%|██████████| 50/50 [14:12<00:00, 17.05s/it]

Answer: 총사업비 관리제도는 2년 이상이 소요되는 대규모 사업에 대해 사업 규모 및 사업 기간을 미리 정하여 관리하는 제도로, 건축 ･토목 사업은 1994년부터 관리하고 있으며, 정보화 사업은 2010년부터, 연구개발(R&D)사업은 2015년부터 관리합니다.
GT: 총사업비 관리제도는 2년 이상이 소요되는 대규모 사업에 대해 사업 규모 및 사업 기간을 미리 정하여 관리하는 제도입니다. 건축・토목 사업은 1994년부터, 정보화 사업은 2010년부터, 연구개발(R&D)사업은 2015년부터 관리하고 있습니다.

{'average_precision': 0.7547155360267217, 'average_recall': 0.8392080684490059, 'average_f1_score': 0.7636748977599865}


,Answer,GT
0,"2024년 기준으로 중앙정부 재정체계는 일반회계 1개, 특별회계 21개, 기금 68...","2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 특별회계 81.7조원으로...","2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,"기금은 예산과 구분되는 재정체계로, 2024년 기준으로 일반회계 1개, 특별회계 2...","기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,일반회계는 특정 사업 운영 및 특정 세입으로 특정 목적의 지출을 충당하는데 사용되며...,"일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,"2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216...","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."


### 'average_f1_score'가 0.68에서 0.76까지 오름

파인튜닝하니 Trainset에 대해서는 성능이 꽤 오름 

In [10]:
config = {
    "llm_predictor": {
        "model_name": "llama3.1",  
        "temperature": 0
    },
    
    "embed_model": "intfloat/multilingual-e5-small",  # "intfloat/multilingual-e5-small",
    
    "text_split":{
        'chunk_size': 1024,
        'chunk_overlap': 64
    },
    
    "save_data_path": './train_data',
    
    
    "search_type": "mmr",           # "mmr"  similarity   similarity_score_threshold
    'search_kwargs_k': 3,           # 유사도 기반 상위 k개
    'search_kwargs_lambda': 0.5,    # 0~1, 0에 가까울수록 다양성, 1에 가까울수록 관련성
    'score_threshold': 0.4,         # 0~1, 쿼리 문장과 최소한 0.x 이상의 유사도를 가진 문서만
    
    'bm25_k' : 3,                   # 검색어 기반 상위 k개
    "ensemble_search_type": "mmr",  # 앙상블 서칭 타입
    "ensemble_weight": [0.4, 0.6],   # bm25와 vector 가중치
    


}

In [11]:
pipeline = Ragpipeline(f'train_data/{reversed_file_mapping[source]}', config)

# 결과를 저장할 리스트 초기화
results = []

for _, row in tqdm(target_df.iterrows(), total=len(target_df), desc=f"Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_path(row['Source'])
    
    # 질문 
    question = row['Question']
    # print(f"Question: {question}")
    
    # 대답
    full_response = pipeline.answer_generation(question)
    first_line_response = full_response.split("\n")[0]
    # print(f"Answer: {first_line_response}")
    
    # 정답 
    # print(f"GT: {row['Answer']}\n")
    
    # 결과 저장
    results.append({
        "Answer": first_line_response,
        "GT": row['Answer']
    })
    
# results 리스트를 DataFrame으로 변환
df_results = pd.DataFrame(results)
pred = df_results['Answer']
gt = df_results['GT']

f1_score = calculate_average_f1_score(gt, pred)
print(f1_score)

# 데이터프레임 출력
df_results.head()

Answering Questions: 100%|██████████| 50/50 [06:44<00:00,  8.09s/it]

{'average_precision': 0.6580799879501459, 'average_recall': 0.7863767863942548, 'average_f1_score': 0.6804873475154852}


,Answer,GT
0,"2024년 기준으로 중앙정부 재정체계는 일반회계 1개, 특별회계 21개, 기금 68...","2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 특별회계 81.7조원으로...","2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,"기금은 일반회계와 구분되는 재정수단으로서, 재정운영의 신축성을 기할 필요가 있을 때...","기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,"일반회계는 국가 고유의 일반적 재정으로 특정 사업 운영을 위해 사용되며, 세입과 세...","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,"2024년도 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 21...","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."


In [12]:
config = {
    "llm_predictor": {
        "model_name": "llama3.1",  
        "temperature": 0
    },
    
    "embed_model": "intfloat/multilingual-e5-base",  # "intfloat/multilingual-e5-small",
    
    "text_split":{
        'chunk_size': 512,
        'chunk_overlap': 64
    },
    
    "save_data_path": './train_data',
    
    
    "search_type": "mmr",           # "mmr"  similarity   similarity_score_threshold
    'search_kwargs_k': 3,           # 유사도 기반 상위 k개
    'search_kwargs_lambda': 0.5,    # 0~1, 0에 가까울수록 다양성, 1에 가까울수록 관련성
    'score_threshold': 0.4,         # 0~1, 쿼리 문장과 최소한 0.x 이상의 유사도를 가진 문서만
    
    'bm25_k' : 3,                   # 검색어 기반 상위 k개
    "ensemble_search_type": "mmr",  # 앙상블 서칭 타입
    "ensemble_weight": [0.4, 0.6],   # bm25와 vector 가중치
    


}

In [13]:
pipeline = Ragpipeline(f'train_data/{reversed_file_mapping[source]}', config)

# 결과를 저장할 리스트 초기화
results = []

for _, row in tqdm(target_df.iterrows(), total=len(target_df), desc=f"Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_path(row['Source'])
    
    # 질문 
    question = row['Question']
    # print(f"Question: {question}")
    
    # 대답
    full_response = pipeline.answer_generation(question)
    first_line_response = full_response.split("\n")[0]
    # print(f"Answer: {first_line_response}")
    
    # 정답 
    # print(f"GT: {row['Answer']}\n")
    
    # 결과 저장
    results.append({
        "Answer": first_line_response,
        "GT": row['Answer']
    })
    
# results 리스트를 DataFrame으로 변환
df_results = pd.DataFrame(results)
pred = df_results['Answer']
gt = df_results['GT']

f1_score = calculate_average_f1_score(gt, pred)
print(f1_score)

# 데이터프레임 출력
df_results.head()

Answering Questions: 100%|██████████| 50/50 [06:40<00:00,  8.01s/it]

{'average_precision': 0.6580799879501459, 'average_recall': 0.7863767863942548, 'average_f1_score': 0.6804873475154852}


,Answer,GT
0,"2024년 기준으로 중앙정부 재정체계는 일반회계 1개, 특별회계 21개, 기금 68...","2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 특별회계 81.7조원으로...","2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,"기금은 일반회계와 구분되는 재정수단으로서, 재정운영의 신축성을 기할 필요가 있을 때...","기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,"일반회계는 국가 고유의 일반적 재정으로 특정 사업 운영을 위해 사용되며, 세입과 세...","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,"2024년도 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 21...","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."


In [14]:
config = {
    "llm_predictor": {
        "model_name": "llama3.1",  
        "temperature": 0
    },
    
    "embed_model": "intfloat/multilingual-e5-base",  # "intfloat/multilingual-e5-small",
    
    "text_split":{
        'chunk_size': 1024,
        'chunk_overlap': 64
    },
    
    "save_data_path": './train_data',
    
    
    "search_type": "mmr",           # "mmr"  similarity   similarity_score_threshold
    'search_kwargs_k': 3,           # 유사도 기반 상위 k개
    'search_kwargs_lambda': 0.5,    # 0~1, 0에 가까울수록 다양성, 1에 가까울수록 관련성
    'score_threshold': 0.4,         # 0~1, 쿼리 문장과 최소한 0.x 이상의 유사도를 가진 문서만
    
    'bm25_k' : 3,                   # 검색어 기반 상위 k개
    "ensemble_search_type": "mmr",  # 앙상블 서칭 타입
    "ensemble_weight": [0.3, 0.7],   # bm25와 vector 가중치
    


}

In [10]:
pipeline = Ragpipeline(f'train_data/{reversed_file_mapping[source]}', config)

# 결과를 저장할 리스트 초기화
results = []

for _, row in tqdm(target_df.iterrows(), total=len(target_df), desc=f"Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_path(row['Source'])
    
    # 질문 
    question = row['Question']
    # print(f"Question: {question}")
    
    # 대답
    full_response = pipeline.answer_generation(question)
    first_line_response = full_response.split("\n")[0]
    # print(f"Answer: {first_line_response}")
    
    # 정답 
    # print(f"GT: {row['Answer']}\n")
    
    # 결과 저장
    results.append({
        "Answer": first_line_response,
        "GT": row['Answer']
    })
    
# results 리스트를 DataFrame으로 변환
df_results = pd.DataFrame(results)
pred = df_results['Answer']
gt = df_results['GT']

f1_score = calculate_average_f1_score(gt, pred)
print(f1_score)

# 데이터프레임 출력
df_results.head()

TypeError: get_embedding() missing 1 required positional argument: 'config'

In [16]:
config = {
    "llm_predictor": {
        "model_name": "llama3.1",  
        "temperature": 0
    },
    
    "embed_model": "intfloat/multilingual-e5-base",  # "intfloat/multilingual-e5-small",
    
    "text_split":{
        'chunk_size': 1024,
        'chunk_overlap': 64
    },
    
    "save_data_path": './train_data',
    
    
    "search_type": "similarity",           # "mmr"  similarity   similarity_score_threshold
    'search_kwargs_k': 5,           # 유사도 기반 상위 k개
    'search_kwargs_lambda': 0.5,    # 0~1, 0에 가까울수록 다양성, 1에 가까울수록 관련성
    'score_threshold': 0.4,         # 0~1, 쿼리 문장과 최소한 0.x 이상의 유사도를 가진 문서만
    
    'bm25_k' : 3,                   # 검색어 기반 상위 k개
    "ensemble_search_type": "mmr",  # 앙상블 서칭 타입
    "ensemble_weight": [0.4, 0.6],   # bm25와 vector 가중치
    


}

pipeline = Ragpipeline(f'train_data/{reversed_file_mapping[source]}', config)

# 결과를 저장할 리스트 초기화
results = []

for _, row in tqdm(target_df.iterrows(), total=len(target_df), desc=f"Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_path(row['Source'])
    
    # 질문 
    question = row['Question']
    # print(f"Question: {question}")
    
    # 대답
    full_response = pipeline.answer_generation(question)
    first_line_response = full_response.split("\n")[0]
    # print(f"Answer: {first_line_response}")
    
    # 정답 
    # print(f"GT: {row['Answer']}\n")
    
    # 결과 저장
    results.append({
        "Answer": first_line_response,
        "GT": row['Answer']
    })
    
# results 리스트를 DataFrame으로 변환
df_results = pd.DataFrame(results)
pred = df_results['Answer']
gt = df_results['GT']

f1_score = calculate_average_f1_score(gt, pred)
print(f1_score)

# 데이터프레임 출력
df_results.head()

Answering Questions: 100%|██████████| 50/50 [06:14<00:00,  7.49s/it]

{'average_precision': 0.5505237362930605, 'average_recall': 0.6800649599870562, 'average_f1_score': 0.5782879714396393}


,Answer,GT
0,"2024년 기준으로 중앙정부 재정체계는 일반회계 1개, 특별회계 21개, 기금 68...","2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,"2024년 중앙정부의 예산 총지출은 656.6조원이며, 일반회계는 356.5조원으로...","2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,"기금은 예산과 구분되는 재정 수단으로서, 재정 운영의 신축성을 기할 필요가 있을 때...","기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,"일반회계는 중앙정부가 운영하는 일반적인 예산이며, 특별회계는 특정 분야나 목적을 위...","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,"2024년 총수입은 1조 5733억원입니다. 이 중 예산수입은 3565억원이며, 기...","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."
